<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#B.-Risiko-und-Renditen" data-toc-modified-id="B.-Risiko-und-Renditen-1">B. Risiko und Renditen</a></span></li><li><span><a href="#B.2-Erwartete-Renditen:-Das-CAPM" data-toc-modified-id="B.2-Erwartete-Renditen:-Das-CAPM-2">B.2 Erwartete Renditen: Das CAPM</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Renditezerlegung" data-toc-modified-id="Renditezerlegung-2.0.1">Renditezerlegung</a></span><ul class="toc-item"><li><span><a href="#Beispiel" data-toc-modified-id="Beispiel-2.0.1.1">Beispiel</a></span></li></ul></li><li><span><a href="#Mehr-zum-CAPM" data-toc-modified-id="Mehr-zum-CAPM-2.0.2">Mehr zum CAPM</a></span></li><li><span><a href="#Ex-Post-und-Ex-Ante-CAPM" data-toc-modified-id="Ex-Post-und-Ex-Ante-CAPM-2.0.3">Ex Post und Ex Ante CAPM</a></span></li><li><span><a href="#CAPM-und-Relevanz-für-aktive-Manager" data-toc-modified-id="CAPM-und-Relevanz-für-aktive-Manager-2.0.4">CAPM und Relevanz für aktive Manager</a></span></li><li><span><a href="#CAPM-und-Vorhersagen" data-toc-modified-id="CAPM-und-Vorhersagen-2.0.5">CAPM und Vorhersagen</a></span></li></ul></li></ul></li></ul></div>

# B. Risiko und Renditen 
# B.2 Erwartete Renditen: Das CAPM

### Renditezerlegung

- Das CAPM stützt sich auf zwei Konstrukte: ein Marktportfolio $M$ und Beta $\beta$, welches das Portfolio mit dem Markt verbindet.
- In der Theorie enthält $M$ alle risikobehafteten Assets, aber in der Praxis approximiert man das Marktportfolio in der Regel durch einen breiten wertgewichteten Index gehandelter inländischer Aktien.

Angenommen, wir haben ein Portfolio $P$ mit Überschussrendite $r_{p}$ und das Marktportfolio $M$ mit Überschussrendite $r_{m}$. Dann ist das Beta des Portfolios $P$ gleich:

$$ \beta_{P} = \frac{\text{Cov}(r_{P},r_{M})}{\text{Var}(r_{M})} $$

Die einfachste Schätzung von $\beta_{P}$ ergibt sich aus einer linearen Regression der Überschussrendite des Portfolios $r_{P}(t)$ auf die Überschussrendite des Marktes $r_{M}(t)$ im gleichen Zeitraum:

$$ r_{P}(t) = \alpha_{P} + \beta_{P}r_{M}(t) + \epsilon_{P}(t)  $$

Dies sind realisierte oder historische Betas und Alphas. Sie können für praktikable Vorhersagen in der Zukunft verwendet werden.

Beta ist eine Möglichkeit, Risiko und Rendite in zwei Teile zu zerlegen: eine Marktkomponente und eine Residualkomponente:


$$ r_{P} = \beta_{P}r_{M} + \theta_{P} $$


Die Residualrendite $\theta_{P}$ ist mit der Marktrendite $r_{M}$ nicht korreliert, folglich gilt für die Varianz des Portfolios $P$:


$$ \sigma_{P}^{2} = \beta^{2}_{P}\sigma_{M}^{2} + \omega_{P}^{2} $$


wobei $\omega_{P}^{2}$ die Residualvarianz des Portfolios ist.

Das **CAPM** besagt, dass die erwartete Residualrendite für alle Aktien und jedes Portfolio gleich Null ist, d.h. $E[\theta_{P}]=0$. Daher wird die erwartete Überrendite des Portfolios $E[r_{P}]=\mu_{P}$ vollständig durch die erwartete Überrendite des Marktes, $E[r_{M}]=\mu_{M}$ und durch das Portfolio $\beta$ bestimmt:

$$ E[r_{P}] = \beta_{P}E[r_{M}] =  \beta_{P}\mu_{M} $$

#### Beispiel

Wir verwenden den S&P500 als "Markt", Macy's (**M**) als Beispiel für eine Aktie und die 1-Monats-Rendite des US Schatzbriefs (Treasury Bill) als risikofreien Zinssatz (wir rechnen den Zinssatz in Tagesrenditen um). Zuerst laden wir die Daten für den Zeitraum 2011-2014. 

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
%matplotlib inline 

Zuerst laden wir die Daten für den risikofreien Zinssatz (aus der Datenbank Fred - Federal Reserve Economic Data) und rechnen ihn in stetige Tageszinsen um:

In [ ]:
one_mon = DataReader('DGS1MO', 'fred',datetime(2012,1,1), datetime(2015,1,1))
one_day = np.log(1+one_mon)/365

Dann laden und plotten wir die S&P500 Tagesrenditen.

In [ ]:
spy = DataReader('SPY',  'yahoo', datetime(2012,1,1), datetime(2015,1,1))
spy_returns = pd.DataFrame(np.diff(np.log(spy['Adj Close'].values)))
spy_returns.index = spy.index.values[1:spy.index.values.shape[0]]
spy_returns.columns = ['S&P500 Returns']

plt.figure(figsize=(15,5));
plt.plot(spy_returns);
plt.ylabel('Returns');
plt.title('S&P500 Returns');
plt.show();

Wir berechnen die Rendite einer Beispielaktie, die in diesem Index gelistet ist - zum Beispiel Macy's (**M**):

In [ ]:
m = DataReader('M',  'yahoo', datetime(2012,1,1), datetime(2015,1,1))
m_returns = pd.DataFrame(np.diff(np.log(m['Adj Close'].values)))
m_returns.index = spy.index.values[1:m.index.values.shape[0]]
m_returns.columns = ["Macy's Returns"]

plt.figure(figsize=(15,5));
plt.plot(m_returns);
plt.ylabel('Returns');
plt.title("Macy's Returns");
plt.show();

Als nächstes müssen wir die Zeitreihen zusammenführen, um sicherzustellen, dass sie im gleichen Zeitraum liegen und dann die Überrenditen berechnen.

In [ ]:
returns = pd.concat([one_day,m_returns,spy_returns],axis=1).dropna()
excess_m = returns["Macy's Returns"].values - returns['DGS1MO'].values
excess_spy = returns["S&P500 Returns"].values - returns['DGS1MO'].values
final_returns = pd.DataFrame(np.transpose([excess_m,excess_spy, returns['DGS1MO'].values]))
final_returns.columns=["Excess Macy's Returns","Excess S&P500 Returns","Risk-free rate"]
final_returns.index = returns.index

plt.figure(figsize=(15,5))
plt.title("Returns")
x = plt.plot(final_returns);
plt.legend(iter(x), final_returns.columns);

Nun werden wir eine lineare Regression unter Verwendung der beiden Zeitreihen durchführen.

In [ ]:
exog = excess_spy
exog

In [ ]:
exog = sm.add_constant(exog, prepend=True)
exog

In [ ]:
import statsmodels.api as sm

exog = excess_spy
exog = sm.add_constant(exog, prepend=True)
mod = sm.OLS(excess_m, exog)
res = mod.fit()
print(res.summary())

Es resultieren die Werte $\beta_{M}=1,0324$ und $\alpha=0,0003$, wobei Letzterer statistisch nicht signifikant ist - im Einklang mit dem CAPM, welches keine risikoadjustierten Überrenditen vorhersagt.

Anhand dieser Ergebnisse können wir die Überschussrenditen in die Marktkomponente und die Residualkomponente aufteilen:

In [ ]:
market_c = excess_spy*1.0324
residual_c = excess_m - market_c

plt.figure(figsize=(15,8))
plt.subplot(211)
plt.title("Market component of excess returns");
plt.plot(final_returns.index,market_c);
plt.subplot(212)
plt.title("Residual component of excess returns");
plt.plot(final_returns.index,residual_c);

### Mehr zum CAPM

Die "sinnvollen" Implikationen des CAPM:

- Sofern das CAPM gilt, handelt es sich bei jedem Portfolio das vom Markt abweicht um ein Nullsummenspiel. Es verfügt über zusätzliches Risiko ohne zusätzliche Renditeerwartungen.
- Wörtlich genommen impliziert das CAPM, dass passives Management optimal ist. Das Eingehen von Residualrisiko wird nicht in Form höherer Renditeerwartungen entlohnt, da es durch Diversifikation vermeidbar ist.


Implikationen für das Portfoliomanagement:

- Nimmt man die erwarteten Renditen aus dem CAPM und steckt sie in einen $\mu$-$\sigma$ Optimierer, erhält man das Marktportfolio.
- Alternativ können wir vom Marktportfolio ausgehen und annehmen, dass es optimal ist, und dann die erwarteten Renditen, die mit dieser Annahme übereinstimmen, herausrechnen.

### Ex Post und Ex Ante CAPM

Erweitern wir die Analyse um 2 weitere Aktien **AMZN** und **UAA** (Amazon und Under Armor):

In [ ]:
a = DataReader('AMZN',  'yahoo', datetime(2012,1,1), datetime(2015,1,1))
a_returns = pd.DataFrame(np.diff(np.log(a['Adj Close'].values)))
a_returns.index = a.index.values[1:a.index.values.shape[0]]
a_returns.columns = ["Amazon Returns"]

ua = DataReader('UAA',  'yahoo', datetime(2012,1,1), datetime(2015,1,1))
ua_returns = pd.DataFrame(np.diff(np.log(ua['Adj Close'].values)))
ua_returns.index = ua.index.values[1:ua.index.values.shape[0]]
ua_returns.columns = ["Under Armor Returns"]

returns = pd.concat([one_day,a_returns,ua_returns],axis=1).dropna()
excess_a = returns["Amazon Returns"].values - returns['DGS1MO'].values
excess_ua = returns["Under Armor Returns"].values - returns['DGS1MO'].values


Wir benötigen die Betas für beide Aktien:

In [ ]:
mod = sm.OLS(excess_a, exog)
res = mod.fit()
print(res.summary())

mod = sm.OLS(excess_ua, exog)
res = mod.fit()
print(res.summary())

Wir berechnen nun die erwarteten Jahresrenditen für unsere drei Aktien und den Markt und zeichnen die ex ante **Security Market Line (SML)**:

In [ ]:
expected_spy = np.log(1+np.mean(excess_spy))*365
betas = np.array([0,1,1.0324,1.2319,1.2590])
expected_rf = np.log(1+np.mean(returns['DGS1MO'].values))*365
expected_returns = betas*expected_spy + expected_rf
data_names = ['Risk-free rate', 'Market','Macys','Under Armor','Amazon']

plt.figure(figsize=(15,5))
plt.plot(betas,expected_returns);
plt.plot(betas,expected_returns,'bo')

for i, txt in enumerate(expected_returns):
    plt.annotate(data_names[i], (betas[i],expected_returns[i]))

plt.xlabel("Beta")
plt.ylabel("Expected Annual Return")

Hier ist der Achsenabschnitt der risikofreie Zinssatz und die Steigung entspricht der erwarteten Überrendite für den Markt. Betrachten wir nun die Ex-Post-Renditen für das nächste Jahr. Der Einfachheit halber nehmen wir an, dass die Betas und der risikofreie Zinssatz gleich bleiben:

In [ ]:
a_new = DataReader('AMZN',  'yahoo', datetime(2015,1,2), datetime(2016,1,1))
a_new_price = a_new['Adj Close'].values
a_annual = (a_new_price[a_new_price.shape[0]-1]/a_new_price[0] - 1)
      
m_new = DataReader('M',  'yahoo', datetime(2015,1,2), datetime(2016,1,1))
m_new_price = m_new['Adj Close'].values
m_annual = (m_new_price[m_new_price.shape[0]-1]/m_new_price[0] - 1)

ua_new = DataReader('UAA',  'yahoo', datetime(2015,1,2), datetime(2016,1,1))
ua_new_price = ua_new['Adj Close'].values
ua_annual = (ua_new_price[ua_new_price.shape[0]-1]/ua_new_price[0] - 1)

spy_new = DataReader('SPY',  'yahoo', datetime(2015,1,2), datetime(2016,1,1))
spy_new_price = spy_new['Adj Close'].values
spy_annual = (spy_new_price[spy_new_price.shape[0]-1]/spy_new_price[0] - 1)

expected_returns_real = [expected_returns[0],spy_annual,m_annual,ua_annual,a_annual]
sml_real = np.array([0,1,1.0324,1.2319,1.2590])*spy_annual
sml_real[0] = expected_returns[0]

In [ ]:

print(expected_returns)
print(sml_real)

In [ ]:
plt.figure(figsize=(15,10))

plt.plot(betas,expected_returns,label='Ex-ante security market line')
plt.plot(betas,expected_returns,'bo')

plt.plot(betas,expected_returns_real,'go', label='Realized Returns')

plt.plot(betas,sml_real,c='r',label='Ex-post security market line')
plt.plot(betas,sml_real,'ro')  

for i, txt in enumerate(expected_returns):
    plt.annotate(data_names[i], (betas[i],expected_returns[i]))
  
for i, txt in enumerate(expected_returns):
    plt.annotate(data_names[i], (betas[i],sml_real[i]))    
    
for i, txt in enumerate(expected_returns):
    plt.annotate(data_names[i], (betas[i],expected_returns_real[i]))
    
plt.xlabel("Beta");
plt.ylabel("Expected Annual Return");
plt.legend(loc=2);

Die Ex-post-SML sagt uns die Renditekomponente, die das CAPM prognostiziert hätte, wenn wir gewusst hätten, wie sich das Marktportfolio entwickeln würde. Die Steigung ist negativ, da der Markt in diesem Zeitraum weniger als den risikofreien Zinssatz verdient hat.

Eine weitere Möglichkeit, das CAPM zu visualisieren, ist die Darstellung der täglichen Überrenditen einer Aktie und des Marktes. Hier stellen wir für den Zeitraum 2012-2014 die Überrenditen für Amazon und S&P500 gegenüber:

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(excess_a,excess_spy,'.')
plt.grid(True)
plt.ylabel("S&P500 Excess Returns");
plt.xlabel("Amazon Excess Returns");

In [ ]:
sns.lmplot(x = "excess_a", y = "excess_spy", 
            data = pd.DataFrame(data = np.transpose([excess_a, excess_spy]), columns = ["excess_a", "excess_spy"]))


### CAPM und Relevanz für aktive Manager

Das CAPM funktioniert gut im Vergleich zu der "naiven" Hypothese, dass die erwarteten Renditen für alle Aktien gleich sind, aber weniger gut, wenn wir die Hypothese testen, dass $\alpha$ 0 ist. Was die Relevanz für aktive Manager betrifft, so sieht das CAPM oberflächlich betrachtet angesichts seiner Implikationen nicht besonders nützlich aus, aber es ist immer noch sinnvoll als Benchmark, anhand derer der Mehrwert des aktiven Managers bewertet werden kann. 

Das CAPM kann einem aktiven Manager auch dabei helfen, zwischen Markt- und Residualkomponenten der Rendite zu unterscheiden. Diese Zerlegung ermöglicht es Managern, das Einnehmen aktiver Marktpositionen zu vermeiden, indem sie Portfoliobetas nahe an 1 wählen (d.h. sie ermöglicht es ihnen, das Risiko des Market Timing zu vermeiden und ihr Augenmerk auf Residualrenditen zu richten, die einen Erwartungswert von Null haben).

### CAPM und Vorhersagen

- Wir benötigen Vorhersagen für die Betas! Am einfachsten ist die Schätzung auf Basis historischer Renditen.
- Wir benötigen Prognosen über erwartete Überrenditen des Marktportfolios! Auch hier sind historisch-basierte Schätzungen die einfachste Option.